In [39]:
import pandas as pd
import geopandas as gpd
import vaex
import glob
import gc
import metadata
import requests
import datetime
# import concurrent.futures
from multiprocessing import Pool

In [40]:
CNAES_alimentacao = [
    "4711301",
    "4711302",
    "4712100",
    "4721102",
    "4721103",
    "4721104",
    "4722901",
    "4722902",
    "4724500",
    "4729602",
    "4729699",
    "5611201",
    "5611202",
    "5611203",
    "5612100",
    "5620103",
    "5620104"
] 
# Segundo o Estudo_Técnico_Mapeamento_Desertos_Alimentares
# https://aplicacoes.mds.gov.br/sagirmps/noticias/arquivos/files/Estudo_T%C3%A9cnico_Mapeamento_Desertos_Alimentares_Final_30_10_2019(1).pdf

In [41]:
df_estabelecimentos = vaex.open('data/brasil_cnpj.hdf5')

In [42]:
df_estabelecimentos.shape

(55813191, 30)

In [43]:
df_estabelecimentos.logradouro_tipo.value_counts()

missing         916949
RUA           36640622
AVENIDA       10430240
ESTRADA        1121829
RODOVIA         906033
                ...   
3  ALTO              1
5  BECO              1
2 PARALELA           1
VIELA PART           1
T�NEL                1
Length: 398, dtype: int64

In [44]:
df_municipios = vaex.read_csv('data/Municipios.zip',
                    compression='zip', 
                    sep=';', 
                    encoding_errors='replace',
                    names=['cd_municipio', 'nome_municipio'])

In [45]:
## TODO
# preencher municipio
# df_municipios.cd_municipio.str.pad(4, side='right', fillchar='0')
df_municipios['cd_municipio_str'] = df_municipios.cd_municipio.to_string().str.pad(4, side='left', fillchar='0')

In [46]:
df_estabelecimentos.shape

(55813191, 30)

In [47]:
df_estabelecimentos = df_estabelecimentos.join(df_municipios, left_on='logradouro_cod_municipio', right_on='cd_municipio_str')

In [48]:
df_alimentacao_br = df_estabelecimentos[df_estabelecimentos.cnae_principal.isin(CNAES_alimentacao)]

In [49]:
df_alimentacao_br['endereco'] = df_alimentacao_br.logradouro_tipo + ' ' + df_alimentacao_br.logradouro_nome + ', ' + df_alimentacao_br.logradouro_numero + ', ' + df_alimentacao_br['nome_municipio'] + ', ' + df_alimentacao_br['logradouro_uf']

In [50]:
df_alimentacao_br['cnpj'] = df_alimentacao_br.cnpj_basico + '/' + df_alimentacao_br.cnpj_ordem + '-' + df_alimentacao_br.cnpj_dv

In [51]:
df_alimentacao_br = df_alimentacao_br[df_alimentacao_br.endereco.str.len() > 5]

In [52]:
df_alimentacao_br.shape

(7287236, 35)

In [74]:
def locate(cnpj, endereco):
    params = {
        'size':'1', 
        'text': endereco,
    }
    
    geolocator = requests.get(url='http://localhost:4000/v1/search', params=params)
    
    if geolocator.json()['features']:
        long = geolocator.json()['features'][0]['geometry']['coordinates'][0]
        lat = geolocator.json()['features'][0]['geometry']['coordinates'][1]
        confidence = geolocator.json()['features'][0]['properties']['confidence']
        return cnpj, long, lat, confidence
    else:
        return None
    

In [75]:
list(zip(df_alimentacao_br[:5].cnpj.tolist(), df_alimentacao_br[:5].endereco.tolist()))

[('41073073/0001-90', 'PRACA DOM EXPEDITO LOPES, 28, CACHOEIRINHA, PE'),
 ('04004834/0001-26', 'RUA BARAO SAO DOMINGOS, SN, MANAUS, AM'),
 ('04006281/0001-40', 'RUA SAO BENTO, 1072, ARARAQUARA, SP'),
 ('41085093/0001-81',
  'AVENIDA AGOSTINHO SILVESTRE, 52, JABOATAO DOS GUARARAPES, PE'),
 ('81687634/0001-66', 'AVENIDA SENADOR SOUZA NAVES, SN, CRUZEIRO DO SUL, PR')]

In [76]:
locate('41073073/0001-90', 'PRACA DOM EXPEDITO LOPES, 28, CACHOEIRINHA, PE')

('41073073/0001-90', -36.2362169, -8.4877925, 0.8)

In [82]:
# cnpj, lat, long, confidence = [], [], [], []
now = datetime.datetime.now()
futures = []

with Pool(12) as p:
    seila = p.starmap(locate, zip(df_alimentacao_br[:1000000].cnpj.tolist(), df_alimentacao_br[:1000000].endereco.tolist()))


# with concurrent.futures.ProcessPoolExecutor(max_workers=11) as executor:
#     for i, row in df_alimentacao_br[:5000].iterrows():
#         futures.append(executor.submit(locate(row, i)))

    

    # break

In [83]:
len(seila)

1000000

In [7]:
{'cnpj':cnpj, 'long':long, 'lat':lat, 'confidence':confidence}

10

In [2]:
# geolocator = Pelias(domain='http://localhost:4000/v1')

In [5]:
# geolocator.geocode('Av. Paulista, 900, São Paulo, SP')

In [7]:
df_estabelecimentos_sp = df_estabelecimentos[df_estabelecimentos.logradouro_cod_municipio == '7107']

In [40]:
df_estabelecimentos_sp.shape

(4934682, 30)

In [19]:
df_estab_alimentacao_sampa = df_estabelecimentos_sp[df_estabelecimentos_sp.cnae_principal.isin(CNAES_alimentacao)]

In [20]:
df_estab_alimentacao_sampa

#,cnpj_basico,cnpj_ordem,cnpj_dv,matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_no_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,logradouro_tipo,logradouro_nome,logradouro_numero,logradouro_complemento,logradouro_bairro,logradouro_cep,logradouro_uf,logradouro_cod_municipio,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
0,61745626,0001,11,1,--,08,20081231,71,--,--,19891013,4721102,--,RUA,SAO POMPONIO,298,A,ALTO DA MOOCA,03189210,SP,7107,--,--,--,--,--,--,--,--,--
1,61901229,0001,91,1,--,02,20051103,00,--,--,19680108,4729699,--,RUA,PANTOJO,1026,--,AGUA RAZA,03343000,SP,7107,--,--,--,--,--,--,--,--,--
2,61903282,0001,21,1,--,08,20150209,73,--,--,19891110,5611201,--,ESTRADA,DO CAMPO LIMPO,3244,--,CAMPO LIMPO,05787001,SP,7107,--,--,--,--,--,--,--,--,--
3,61987707,0001,28,1,JOSE CAMPOY SERRANO BAR,08,20081231,71,--,--,19730716,5611203,--,AVENIDA,CARLOS LIVIERO,263,--,VL LIVIGRO,04186100,SP,7107,--,--,--,--,--,--,--,--,--
4,62055264,0001,08,1,--,08,20081231,71,--,--,19681106,5611203,--,AVENIDA,BRIG LUIS ANTONIO,1605,LOJA,BELA VISTA,01317002,SP,7107,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"526,920",14752332,0001,37,1,--,08,20180201,67,--,--,20111212,4712100,"4729699,4723700,5620104",RUA,BISPO ISAIAS F SUCASAS,208,CASA 2,JARDIM MATARAZZO,03814060,SP,7107,11,87342117,--,--,--,--,joaomariamiranda@r7.com,--,--
"526,921",14752977,0001,70,1,COMIDA DO BEM CONGELADOS,08,20150212,01,--,--,20111212,5620104,--,RUA,BRANCO DE MORAIS,633,--,CHACARA SANTO ANTONIO (ZONA SUL),04718010,SP,7107,11,35356244,--,--,--,--,sac@comidadobem.com.br,--,--
"526,922",14756428,0001,73,1,LANCHONETE ARAUJO,08,20180201,67,--,--,20111213,5611203,"5620104,4729699",RUA,SAO TEODORO,838,--,VILA CARMOSINA,08290005,SP,7107,11,25244211,--,--,--,--,pessoas@r7.com,--,--
"526,923",14756813,0001,10,1,--,02,20111213,00,--,--,20111213,4712100,--,RUA,EUGENIO DARODES,190,--,JARDIM MABEL,08121620,SP,7107,11,84196054,--,--,--,--,--,--,--


In [27]:
df_estab_alimentacao_sampa['cnpj'] = df_estab_alimentacao_sampa.cnpj_basico + '/' + df_estab_alimentacao_sampa.cnpj_ordem + '-' + df_estab_alimentacao_sampa.cnpj_dv

In [29]:
df_estab_alimentacao_sampa['endereco'] = df_estab_alimentacao_sampa.logradouro_tipo + ' ' + df_estab_alimentacao_sampa.logradouro_nome + ', ' +df_estab_alimentacao_sampa.logradouro_numero


In [33]:
df_estab_alimentacao_sampa.shape

(526925, 32)

In [35]:
df_estab_alimentacao_sampa = df_estab_alimentacao_sampa[df_estab_alimentacao_sampa.endereco.str.len() > 5]

In [39]:
df_estab_alimentacao_sampa.shape

(525771, 32)

In [38]:
df_estab_alimentacao_sampa.export_csv('results/estabelecimentos_alimentacao_sampa.csv')

In [48]:
df_estab_alimentacao_sampa.shape

(525771, 32)

In [45]:
df_estabelecimentos_geocoded = gpd.read_file('results/estabelecimentos_alimentares_v01.gpkg')

In [47]:
df_estabelecimentos_geocoded.shape

(525599, 24)

In [51]:
df_estab_alimentacao_sampa = df_estab_alimentacao_sampa.to_pandas_df()

In [65]:
df_estabelecimentos_geocoded.merge(df_estab_alimentacao_sampa, on='cnpj', how='left', copy=False)

,cnpj,nice_name,label,confidence,accuracy,source,layer,street,housenumber,postalcode,...,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial,endereco
0,61745626/0001-11,Rua Sao Pomponio 298,"Rua Sao Pomponio 298, São Paulo, Brazil",1.0,point,openaddresses,address,Rua Sao Pomponio,298,03189-210,...,None,None,None,None,None,None,None,None,None,"RUA SAO POMPONIO, 298"
1,61901229/0001-91,Rua Pantojo 1026,"Rua Pantojo 1026, São Paulo, Brazil",1.0,point,openaddresses,address,Rua Pantojo,1026,03343-000,...,None,None,None,None,None,None,None,None,None,"RUA PANTOJO, 1026"
2,61903282/0001-21,Estrada Do Campo Limpo 3244,"Estrada Do Campo Limpo 3244, São Paulo, Brazil",1.0,point,openaddresses,address,Estrada Do Campo Limpo,3244,05790-000,...,None,None,None,None,None,None,None,None,None,"ESTRADA DO CAMPO LIMPO, 3244"
3,61987707/0001-28,Avenida Carlos Liviero,"Avenida Carlos Liviero, São Paulo, Brazil",0.8,centroid,openstreetmap,street,Avenida Carlos Liviero,None,None,...,None,None,None,None,None,None,None,None,None,"AVENIDA CARLOS LIVIERO, 263"
4,62055264/0001-08,São Paulo,"São Paulo, Brazil",0.6,centroid,whosonfirst,locality,None,None,None,...,None,None,None,None,None,None,None,None,None,"AVENIDA BRIG LUIS ANTONIO, 1605"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711764,14756428/0001-73,Rua Sao Teodoro 838,"Rua Sao Teodoro 838, São Paulo, Brazil",1.0,point,openaddresses,address,Rua Sao Teodoro,838,08290-000,...,11,25244211,None,None,None,None,pessoas@r7.com,None,None,"RUA SAO TEODORO, 838"
711765,14756813/0001-10,Rua Eugênio Darodes,"Rua Eugênio Darodes, São Paulo, Brazil",0.8,centroid,openstreetmap,street,Rua Eugênio Darodes,None,None,...,11,84196054,None,None,None,None,None,None,None,"RUA EUGENIO DARODES, 190"
711766,14756813/0001-10,Rua Eugênio Darodes,"Rua Eugênio Darodes, São Paulo, Brazil",0.8,centroid,openstreetmap,street,Rua Eugênio Darodes,None,None,...,11,84196054,None,None,None,None,None,None,None,"RUA EUGENIO DARODES, 190"
711767,14756869/0001-75,Avenida Agua Vermelha 120,"Avenida Agua Vermelha 120, São Paulo, Brazil",1.0,point,openaddresses,address,Avenida Agua Vermelha,120,08431-470,...,11,20160060,None,None,None,None,seth@r7.com,None,None,"AVENIDA AGUA VERMELHA, 120"
